In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as smm

In [2]:
## Load the reference file (classification for each sample)
ref = pd.read_csv("../data/pcaa-master-platinum_set_2020.tsv", sep="\t", header=0, usecols=[0,7,8,12,13,28,32])
print(ref.shape)
ref.head()

(3212, 7)


,study,patient_barcode,sample_barcode,min_cnv,number_amp_region,sample_classification_nam_2019,sample_classification
0,PCWG,DO51086,SA530559,4,30,NaN,Non-circular
1,PCWG,DO48759,SA515463,4,5,NaN,Non-circular
2,PCWG,DO48916,SA517420,4,0,NaN,No SCNA detected
3,PCWG,DO51046,SA530430,4,2,NaN,No SCNA detected
4,PCWG,DO45251,SA501507,4,3,NaN,Circular


In [3]:
## Only keep TCGA samples
ref = ref[ref['study'] == 'TCGA']
print(ref.shape)
ref.head()

(1921, 7)


,study,patient_barcode,sample_barcode,min_cnv,number_amp_region,sample_classification_nam_2019,sample_classification
1291,TCGA,TCGA-02-2483,TCGA-02-2483-01,1,20,Distal,Heavily-rearranged
1292,TCGA,TCGA-02-2485,TCGA-02-2485-01,1,33,Circular,Circular
1293,TCGA,TCGA-04-1331,TCGA-04-1331-01,1,4,Linear,Non-circular
1294,TCGA,TCGA-04-1347,TCGA-04-1347-01,1,12,Circular,Circular
1295,TCGA,TCGA-04-1349,TCGA-04-1349-01,1,26,Distal,Heavily-rearranged


In [23]:
## Load the oncoprint matrix after querying genes against all TCGA studies
res = pd.read_csv("../data/PATIENT_DATA_oncoprint.tsv", sep="\t", header=0, skiprows=[1,2,3,4])
print(res.shape)
res.head()

(315, 10955)


,track_name,track_type,TCGA-ER-A195,TCGA-XK-AAJA,TCGA-CH-5788,TCGA-OR-A5JX,TCGA-CJ-4886,TCGA-EQ-A4SO,TCGA-CE-A3MD,TCGA-DM-A28K,...,TCGA-97-A4M2,TCGA-97-7552,TCGA-97-8552,TCGA-98-A53C,TCGA-98-A53D,TCGA-98-A53H,TCGA-99-AA5R,TCGA-13-2066,TCGA-BP-4345,TCGA-OR-A5OG
0,APC,CNA,homdel_rec,homdel_rec,homdel_rec,Amplification,Amplification,NaN,NaN,homdel_rec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
## Filter samples out - only keep samples/patients with ecDNA classification
res = res[['track_name', 'track_type'] + res.columns[res.columns.isin(ref['patient_barcode'])].tolist()]
print(res.shape)
res.head()

(315, 1902)


,track_name,track_type,TCGA-CH-5788,TCGA-FS-A1ZG,TCGA-DD-A3A8,TCGA-AD-6964,TCGA-A2-A0EY,TCGA-EJ-7784,TCGA-EJ-5531,TCGA-CH-5748,...,TCGA-VD-A8K8,TCGA-VD-A8KA,TCGA-VD-A8KD,TCGA-VD-A8KE,TCGA-VD-A8KF,TCGA-VD-A8KH,TCGA-VD-A8KL,TCGA-VD-A8KN,TCGA-VD-AA8O,TCGA-VD-AA8P
0,APC,CNA,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
ref2 = ref[ref['patient_barcode'].isin(res.columns)]
print(ref2.shape)
ref2.head()

(1900, 7)


,study,patient_barcode,sample_barcode,min_cnv,number_amp_region,sample_classification_nam_2019,sample_classification
1291,TCGA,TCGA-02-2483,TCGA-02-2483-01,1,20,Distal,Heavily-rearranged
1292,TCGA,TCGA-02-2485,TCGA-02-2485-01,1,33,Circular,Circular
1293,TCGA,TCGA-04-1331,TCGA-04-1331-01,1,4,Linear,Non-circular
1294,TCGA,TCGA-04-1347,TCGA-04-1347-01,1,12,Circular,Circular
1295,TCGA,TCGA-04-1349,TCGA-04-1349-01,1,26,Distal,Heavily-rearranged


In [26]:
cnt = Counter(ref2['sample_classification'])
cnt

Counter({'Heavily-rearranged': 190,
         'Circular': 335,
         'Non-circular': 165,
         'No SCNA detected': 1041,
         'BFB': 169})

In [27]:
## Since the mRNA track and Protein track are empty, we delete these data
res = res[res['track_type'] != 'MRNA']
print(res.shape)
res = res[res['track_type'] != 'PROTEIN']
print(res.shape)
res.head()

(252, 1902)
(189, 1902)


,track_name,track_type,TCGA-CH-5788,TCGA-FS-A1ZG,TCGA-DD-A3A8,TCGA-AD-6964,TCGA-A2-A0EY,TCGA-EJ-7784,TCGA-EJ-5531,TCGA-CH-5748,...,TCGA-VD-A8K8,TCGA-VD-A8KA,TCGA-VD-A8KD,TCGA-VD-A8KE,TCGA-VD-A8KF,TCGA-VD-A8KH,TCGA-VD-A8KL,TCGA-VD-A8KN,TCGA-VD-AA8O,TCGA-VD-AA8P
0,APC,CNA,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
## Strategy 3: ecDNA+ = Circular; ecDNA- = Non-circular
ecdna_pos = ref[ref['sample_classification'] == 'Circular']['patient_barcode']
ecdna_neg = ref[ref['sample_classification'] == 'Non-circular']['patient_barcode']
pos = res.columns[res.columns.isin(ecdna_pos)].tolist()
neg = res.columns[res.columns.isin(ecdna_neg)].tolist()
n_pos, n_neg = len(pos), len(neg)
print("Oncoprint: {}, {}, {}".format(pos[-1], n_pos, n_neg))
cols_met = ['track_name', 'track_type'] + pos + neg
df = res[cols_met]
print(df.shape)
df.head()

Oncoprint: TCGA-G2-A2EK, 335, 165
(189, 502)


,track_name,track_type,TCGA-D3-A2JC,TCGA-DX-A23R,TCGA-05-4402,TCGA-D7-6528,TCGA-A6-2677,TCGA-CV-6948,TCGA-A6-5656,TCGA-BR-4267,...,TCGA-EK-A3GJ,TCGA-FU-A3NI,TCGA-G3-A25W,TCGA-IR-A3LC,TCGA-KL-8328,TCGA-RZ-AB0B,TCGA-V4-A9E7,TCGA-V4-A9EE,TCGA-V4-A9EU,TCGA-V4-A9F3
0,APC,CNA,homdel_rec,Amplification,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
## Save this oncoprint matrix
df.to_csv("../data/oncoprint3.tsv", sep="\t", index=False)

In [29]:
## Convert the oncoprint matrix into a numerical matrix
genes = df['track_name'].unique()
n_gene = len(genes)
print("{} ecDNA+ samples; {} ecDNA- samples; {} genes".format(n_pos, n_neg, n_gene))

335 ecDNA+ samples; 165 ecDNA- samples; 63 genes


In [30]:
L = np.zeros(shape=(n_gene, n_pos+n_neg))
G = np.zeros(shape=(n_gene, n_pos+n_neg))

In [31]:
## List of LoF mutations
loss = ['Truncating mutation (putative driver)', 'Truncating mutation (putative passenger)', \
        'Missense Mutation (putative driver)', 'Inframe Mutation (putative driver)', \
        'Deep Deletion', 'homdel_rec']

## List of GoF mutations
gain = ['Amplification', 'amp_rec']

In [32]:
## Iterate over each genes & ignore FUSION at this moment
for i in range(n_gene):
    df_sel = df[(df['track_name'] == genes[i]) & (df['track_type'] != 'FUSION')]
    for j in range(2, df.shape[1]):
        L[i,j-2] = df_sel[df_sel.columns[j]].isin(loss).any()
        G[i,j-2] = df_sel[df_sel.columns[j]].isin(gain).any()

In [33]:
loss_pos = np.sum(L[:,:n_pos], axis=1)
loss_neg = np.sum(L[:,n_pos:], axis=1)

In [34]:
loss_del = []
for i in range(n_gene):
    if loss_pos[i] == 0 and loss_neg[i] == 0: 
        print(i)
        loss_del.append(i)

4
21
28
39
50
62


In [21]:
gene_loss = np.delete(genes, loss_del)
loss_pos = np.delete(loss_pos, loss_del)
loss_neg = np.delete(loss_neg, loss_del)
len(gene_loss), len(loss_pos), len(loss_neg)

(57, 57, 57)

In [35]:
fisher, fisher_onetail = np.zeros(len(gene_loss)), np.zeros(len(gene_loss))
alpha = 0.05

for i in range(len(gene_loss)):
    contigency_table = np.array([[loss_pos[i], loss_neg[i]], [n_pos - loss_pos[i], n_neg - loss_neg[i]]])
    _, fisher[i] = stats.fisher_exact(contigency_table, alternative="two-sided")
    _, fisher_onetail[i] = stats.fisher_exact(contigency_table, alternative="greater")

In [36]:
rej_fisher, fisher, _, _ = smm.multipletests(fisher, alpha=alpha, method='fdr_bh')
rej_fisher_onetail, fisher_onetail, _, _ = smm.multipletests(fisher_onetail, alpha=alpha, method='fdr_bh')

In [38]:
gene_loss[rej_fisher], gene_loss[rej_fisher_onetail]

(array([], dtype=object), array([], dtype=object))

In [43]:
gain_pos = np.sum(G[:,:n_pos], axis=1)
gain_neg = np.sum(G[:,n_pos:], axis=1)

In [40]:
gain_del = []
for i in range(n_gene):
    if gain_pos[i] == 0 and gain_neg[i] == 0: 
        print(i)
        gain_del.append(i)

30
57


In [44]:
gene_gain = np.delete(genes, gain_del)
gain_pos = np.delete(gain_pos, gain_del)
gain_neg = np.delete(gain_neg, gain_del)
len(gene_gain), len(gain_pos), len(gain_neg)

(61, 61, 61)

In [45]:
fisher_gain, fisher_onetail_gain = np.zeros(len(gene_gain)), np.zeros(len(gene_gain))
alpha = 0.05

for i in range(len(gene_gain)):
    contigency_table = np.array([[gain_pos[i], gain_neg[i]], [n_pos - gain_pos[i], n_neg - gain_neg[i]]])
    _, fisher_gain[i] = stats.fisher_exact(contigency_table, alternative="two-sided")
    _, fisher_onetail_gain[i] = stats.fisher_exact(contigency_table, alternative="greater")

In [46]:
rej_fisher_gain, fisher_gain, _, _ = smm.multipletests(fisher_gain, alpha=alpha, method='fdr_bh')
rej_fisher_onetail_gain, fisher_onetail_gain, _, _ = smm.multipletests(fisher_onetail_gain, alpha=alpha, method='fdr_bh')

In [47]:
gene_gain[rej_fisher_gain], gene_gain[rej_fisher_onetail_gain]

(array([], dtype=object), array([], dtype=object))